<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_rnn_cwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [4]:
uploaded = files.upload()
'''Load data'''
WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))['P']
#WHE_data = pd.read_csv('Electricity_WHE.csv')['P']

Saving Electricity_WHE.csv to Electricity_WHE.csv


In [5]:
uploaded = files.upload()
CWE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CWE.csv']))['P']
#CWE_data = pd.read_csv('Electricity_CWE.csv')['P']

Saving Electricity_CWE.csv to Electricity_CWE.csv


In [0]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

#S0,S1:1-350,S2:350+

cwe_Y_raw = CWE_data[window_size-1:].values.reshape([CWE_data.shape[0]-window_size+1,1])
dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
cwe_Y = np.concatenate([cwe_Y_raw[0:472500,:],cwe_Y_raw[475500:,:]],axis=0)
categorized_cwe_Y = np.ones(cwe_Y.shape)*2
categorized_cwe_Y[[np.where(cwe_Y==0)[0]],:]=0
categorized_cwe_Y[[np.where((cwe_Y>0)&(cwe_Y<=350))[0]],:]=1
#categorized_cwe_Y[[np.where((cwe_Y>350))[0]],:]=2

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cweY_1hot = encoder.fit_transform(categorized_cwe_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX, cweY_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
max_indice = np.argmax(cweY_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1018321
1      26515
2       3305
dtype: int64

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:5000,1:5000,2:2900})
X_train, y_train = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)
#from imblearn.under_sampling import EditedNearestNeighbours
#enn = EditedNearestNeighbours()
#X_train, y_train = enn.fit_resample(X_train, y_train)

#from imblearn.under_sampling import NeighbourhoodCleaningRule
#ncr = NeighbourhoodCleaningRule()
#X_train, y_train = ncr.fit_resample(X_train, y_train)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=512
unit_num=128

In [10]:
'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 3],name='y')

lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num,name='lstm_cell')
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],512,activation=tf.nn.leaky_relu,name='l1')
l2 = tf.layers.dense(l1,1024,activation=tf.nn.leaky_relu,name='l2')
l3 = tf.layers.dense(l2,512,activation=tf.nn.leaky_relu,name='l3')
l4 = tf.layers.dense(l3,256,activation=tf.nn.leaky_relu,name='l4')
l5 = tf.layers.dense(l4,128,activation=tf.nn.leaky_relu,name='l5')
l6 = tf.layers.dense(l5,84,activation=tf.nn.leaky_relu,name='l6')
l7 = tf.layers.dense(l6,64,activation=tf.nn.leaky_relu,name='l7')
l8 = tf.layers.dense(l7,48,activation=tf.nn.leaky_relu,name='l8')
l9 = tf.layers.dense(l8,32,activation=tf.nn.leaky_relu,name='l9')
l10 = tf.layers.dense(l9,24,activation=tf.nn.leaky_relu,name='l10')
l11 = tf.layers.dense(l10,16,activation=tf.nn.leaky_relu,name='l11')
l12 = tf.layers.dense(l11,8,activation=tf.nn.leaky_relu,name='l12')
pred = tf.layers.dense(l12,3,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net_cwe/save_cwe_rnn.ckpt')

flag = False
for j in range(0,1000):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(12800,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-3].reshape([batch_size,window_size,1])
    batch_y = batch_train[:,-3:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
    acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
    print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
    print('Test Acc=%.2f'% acc_test)
    
    pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
    y_lables_argmax = tf.argmax(tf_y,axis=1)  
    y_pred_argmax = tf.argmax(pre,axis=1)
    confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
    #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
    print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
    if acc_train>=0.99:
      print(j)
      flag = True
      break
    if(flag==True):
        print(flag)
        break
    if(j//100==0):
      save_path = saver.save(sess, "my_net_cwe/save_cwe_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from my_net_cwe/save_cwe_rnn.ckpt
###iteration: 0###
train loss= 0.9096, Acc=0.53
Test Acc=0.53
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
[[112  83  15]
 [ 45 115  26]
 [  4  68  44]]
###iteration: 1###
train loss= 0.8638, Acc=0.57
Test Acc=0.53
[[105  50  41]
 [ 43  96  50]
 [  1  35  91]]
###iteration: 2###
train loss= 0.9113, Acc=0.54
Test Acc=0.58
[[109  60  38]
 [ 48 125  45]
 [  6  41  40]]
###iteration: 3###
train loss= 0.9059, Acc=0.57
Test Acc=0.50
[[101  63  26]
 [ 32 121  39]
 [  3  58  69]]
###iteration: 4###
train loss= 0.9221, Acc=0.55
Test Acc=0.35
[[ 74  85  34]
 [  8 122  50]
 [  0  51  88]]
###iteration: 5###
train loss= 0.9363, Acc=0.53
Test Acc=0.39
[[ 71  75  45]
 [ 15 123  54]
 [  1  53  75]]
###iteration: 6###
train loss= 0.8711, Acc=0.57
Test Acc=0.56
[[112  70  13]
 [ 46 123  20]
 [  5 

In [0]:
#!mkdir my_net_cwe
mv checkpoint save_cwe_rnn.ckpt.data-00000-of-00001 save_cwe_rnn.ckpt.index save_cwe_rnn.ckpt.meta my_net_cwe